In [1]:
import random
import json
import pickle
import numpy as np
import nltk

In [2]:
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

In [4]:


lemmatizer = WordNetLemmatizer()
intents = json.loads(open(r"C:\Users\HP\Downloads\ml_model\ml_model\chatbot (2)\intents.json", encoding="utf-8").read())


In [5]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))




In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the input shape (adjust according to your data)
input_shape = (None, 121)  

# Recreate the model architecture
model = Sequential([
    Dense(128, input_shape=input_shape, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(classes), activation='softmax')
])

# Load the weights
model.load_weights('chatbot_model.h5')  # Load only the weights, not the entire model

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [8]:
def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [9]:
def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
    return return_list

In [10]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

In [11]:
print("The Chatbot is running!")

while True:
    message = input("")
    ints = predict_class (message)
    res = get_response (ints, intents)
    print (res)

The Chatbot is running!
1/1 [==============================] - 0s 91ms/step
Hello. Welcome to ABC bank
1/1 [==============================] - 0s 19ms/step
Hi there
1/1 [==============================] - 0s 19ms/step
Welcome
1/1 [==============================] - 0s 22ms/step
Hello. Welcome to ABC bank
